# RDL analytics interface

### Cell > Run all  or   CTRL + Enter

In [ ]:
import ipywidgets as widgets
import requests
import pandas as pd
import numpy as np
import time
from IPython.display import display, clear_output, HTML
from PIL import Image as PILImage
from io import BytesIO
from runAnalysis import run_analysis
from utils import plot_results
import matplotlib.pyplot as plt
%matplotlib inline

# Load the image for the top-right corner
with open("rdl_logo.png", "rb") as img_file:
    img = PILImage.open(img_file)
    buffer = BytesIO()
    img.save(buffer, format="PNG")
    img_data = buffer.getvalue()

##### delete after fixing
# Mock damage functions for demonstration
def mortality_factor(x):
    return 1 - np.exp(-0.5 * x)

def damage_factor_builtup(x):
    return 1 - 1 / (1 + (x/3)**2)

def damage_factor_agri(x):
    return 1 - 1 / (1 + x)
########

# Create UI elements with adjusted label widths and unique IDs

## Country
# Read the countries list file
df = pd.read_csv('countries.csv')

# Create a dictionary mapping country names to ISO_A3 codes
country_dict = dict(zip(df['NAM_0'], df['ISO_A3']))
iso_to_country = dict(zip(df['ISO_A3'], df['NAM_0']))

# Create the Combobox widget with auto-complete functionality
country_selector = widgets.Combobox(
    placeholder='Type country name...',
    options=list(country_dict.keys()),  # Available country options
    description='Country:',
    layout=widgets.Layout(width='300px'),
    ensure_option=True,  # Ensure that only valid options can be selected
)

country_selector_id = f'country-selector-{id(country_selector)}'
country_selector.add_class(country_selector_id)

# Define a function to handle changes in the combobox
def on_country_change(change):
    with output:
        output.clear_output()
        selected_country = change['new']
        if selected_country in country_dict.keys():
            iso_a3 = country_dict[selected_country]
            print(f'Selected Country: {selected_country}, ISO_A3 Code: {iso_a3}')
        else:
            print('Please select a valid country from the list.')

# Attach the function to the combobox
country_selector.observe(on_country_change, names='value')

# Administrative level boundaries
adm_level_selector = widgets.Dropdown(options=[(str(i), i) for i in range(1, 4)], value=1, description='Adm boundaries level:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})
adm_level_selector_id = f'adm-level-selector-{id(adm_level_selector)}'
adm_level_selector.add_class(adm_level_selector_id)

# Hazard
hazard_selector = widgets.Dropdown(options=[
    ('Fluvial Undefended', 'FLUVIAL_UNDEFENDED'), 
    ('Fluvial Defended', 'FLUVIAL_DEFENDED'),
    ('Pluvial Defended', 'PLUVIAL_DEFENDED'), 
    ('Coastal Undefended', 'COASTAL_UNDEFENDED'), 
    ('Coastal Defended', 'COASTAL_DEFENDED')
], value='FLUVIAL_UNDEFENDED', description='Hazard process:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})
hazard_selector_id = f'hazard-selector-{id(hazard_selector)}'
hazard_selector.add_class(hazard_selector_id)

## Hazard threshold
hazard_threshold_slider = widgets.IntSlider(value=20, min=0, max=200, description='Hazard threshold:', layout=widgets.Layout(width='420px'), style={'description_width': '150px'})
hazard_threshold_slider_id = f'hazard-threshold-slider-{id(hazard_threshold_slider)}'
hazard_threshold_slider.add_class(hazard_threshold_slider_id)

## Period
period_selector = widgets.Dropdown(options=['2020', '2030', '2050', '2080'], value='2020', description='Hazard reference period:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})
period_selector_id = f'period-selector-{id(period_selector)}'
period_selector.add_class(period_selector_id)

## Scenario
scenario_selector = widgets.Dropdown(options=[
    ('SSP1-2.6', 'SSP1_2.6'),
    ('SSP2-4.5', 'SSP2_4.5'),
    ('SSP3-7.0', 'SSP3_7.0'),
    ('SSP5-8.5', 'SSP5_8.5')
], value='SSP3_7.0', description='Climate scenario:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})
scenario_selector_id = f'scenario-selector-{id(scenario_selector)}'
scenario_selector.add_class(scenario_selector_id)

## Hazard return periods
return_periods_selector = widgets.SelectMultiple(
    options=[5, 10, 20, 50, 100, 200, 500, 1000],
    value=[5, 10, 20, 50, 100, 200, 500, 1000],
    rows=8,
    description='Return Periods:',
    disabled=False,
    layout=widgets.Layout(width='400px'), style={'description_width': '150px'}
)
return_periods_selector_id = f'return_periods-selector-{id(return_periods_selector)}'
return_periods_selector.add_class(return_periods_selector_id)

# Exposure
exposure_selector = widgets.SelectMultiple(options=[
    ('Population', 'POP'),
    ('Built-up', 'BU'),
    ('Agriculture', 'AGR')
], description='Exposure:', value=['POP'], layout=widgets.Layout(width='400px'), style={'description_width': '150px'})
exposure_selector_id = f'exposure-selector-{id(exposure_selector)}'
exposure_selector.add_class(exposure_selector_id)

## Custom exposure
custom_exposure_container = widgets.VBox([])
custom_exposure_radio = widgets.RadioButtons(options=['Default exposure', 'Custom exposure'], description='', layout=widgets.Layout(width='400px'))
custom_exposure_radio_id = f'custom-exposure-radio-{id(custom_exposure_radio)}'
custom_exposure_radio.add_class(custom_exposure_radio_id)

custom_exposure_textbox = widgets.Text(value='', placeholder='Enter filename (without extension)', layout=widgets.Layout(width='400px'))
custom_exposure_textbox_id = f'custom-exposure-textbox-{id(custom_exposure_textbox)}'
custom_exposure_textbox.add_class(custom_exposure_textbox_id)

# Approach
approach_selector = widgets.Dropdown(options=[('Classes', 'Classes'), ('Function', 'Function')], value='Function', description='Approach:', layout=widgets.Layout(width='400px'), style={'description_width': '150px'})
approach_selector_id = f'approach-selector-{id(approach_selector)}'
approach_selector.add_class(approach_selector_id)

class_edges_table = widgets.VBox()

## Define add_class_button before using it
add_class_button = widgets.Button(description="Add Class", layout=widgets.Layout(width='150px'))

# Create info box
info_box = widgets.Textarea(value='', description='', placeholder='Hover mouse on items for description.', disabled=True, layout=widgets.Layout(height='250px', width='300px'))
info_box.add_class('info-box')

# Function to update scenario dropdown visibility
def update_scenario_visibility(*args):
    scenario_selector.layout.display = 'none' if period_selector.value == '2020' else 'block'

period_selector.observe(update_scenario_visibility, 'value')
update_scenario_visibility()

# Function to update custom exposure textboxes
def update_custom_exposure_textboxes():
    custom_exposure_container.children = []
    if custom_exposure_radio.value == 'Custom exposure':
        for exp in exposure_selector.value:
            textbox = widgets.Text(
                value='',
                placeholder=f'Enter filename for {exp} (without extension)',
                layout=widgets.Layout(width='400px')
            )
            custom_exposure_container.children += (textbox,)

# Update the custom exposure visibility function
def update_custom_exposure_visibility(*args):
    custom_exposure_container.layout.display = 'none' if custom_exposure_radio.value == 'Default exposure' else 'block'
    update_custom_exposure_textboxes()

exposure_selector.observe(update_custom_exposure_textboxes, 'value')
custom_exposure_radio.observe(update_custom_exposure_visibility, 'value')    
    
# Create a new output widget for the preview chart and class edges table
approach_box = widgets.Output(layout=widgets.Layout(width='300px', height='250px'))

# For approach = classes, define class edges
class_edges_table = widgets.VBox()

# For approach = function, preview impact function
def preview_impact_func():
    if approach_selector.value != 'Function':
        return
    
    with approach_box:
        clear_output(wait=True)  # Clear the previous output
        selected_exposures = exposure_selector.value
        if not selected_exposures:
            print("Please select at least one exposure category")            
            return

        steps = np.arange(0, 6, 0.1)
        fig, ax = plt.subplots(figsize=(4, 3))
        
        damage_factors = {
            'POP': mortality_factor,
            'BU': damage_factor_builtup,
            'AGR': damage_factor_agri
        }
        
        for exposure in selected_exposures:
            if exposure in damage_factors:
                damage_factor = damage_factors[exposure]
                _, = ax.plot([damage_factor(x) for x in steps], label=exposure)
            else:
                print(f"Unknown exposure category: {exposure}")
        
        ax.grid(True)
        ax.legend()
        
        label_steps = range(0, len(steps)+10, 10)
        ax.set_xticks(label_steps)
        ax.set_xticklabels([i / 10 for i in label_steps])
        ax.set_xlabel("Hazard intensity")
        ax.set_ylabel("Impact Factor")
        
        plt.tight_layout()
        display(fig)
        plt.close(fig)  # Close the figure to free up memory

# Define add_class_button before using it
add_class_button = widgets.Button(description="Add Class", layout=widgets.Layout(width='150px'))

# Functions to dynamically add class edges
def create_class_row(index):
    delete_button = widgets.Button(description="Delete", layout=widgets.Layout(width='70px'))
    row = widgets.HBox([
        widgets.Label(f'Class {index}:', layout=widgets.Layout(width='100px')),
        widgets.FloatText(value=0.0, description='', layout=widgets.Layout(width='100px')),
        delete_button
    ])
    delete_button.on_click(lambda b: delete_class(row))
    return row

def delete_class(row):
    if len(class_edges_table.children) > 2:  # More than one class row plus the "Add Class" button
        class_edges_table.children = [child for child in class_edges_table.children if child != row]
        # Renumber the remaining classes
        for i, child in enumerate(class_edges_table.children[:-1]):  # Exclude the "Add Class" button
            child.children[0].value = f'Class {i+1}:'
        update_delete_buttons()

def update_delete_buttons():
    for child in class_edges_table.children[:-1]:  # Exclude the "Add Class" button
        display_text = 'block' if len(class_edges_table.children) > 2 else 'none'
        child.children[2].layout.display = display_text

# Define add_class_button
add_class_button = widgets.Button(description="Add Class", layout=widgets.Layout(width='150px'))
add_class_button.on_click(lambda b: add_class(b))

def add_class():
    new_index = len(class_edges_table.children)
    class_edges_table.children = list(class_edges_table.children[:-1]) + [create_class_row(new_index)] + [add_class_button]
    update_delete_buttons()

def update_class_edges_table():
    approach_box.clear_output()
    with approach_box:
        if approach_selector.value == 'Classes':
            # Clear existing table
            class_edges_table.children = []
            # Add initial class rows
            class_edges_table.children = [create_class_row(i) for i in range(1, 4)]
            # Add the "Add Class" button
            class_edges_table.children += (add_class_button,)
            update_delete_buttons()
            display(class_edges_table)
        else:
            preview_impact_func()

    # Ensure at least one class remains
    if len(class_edges_table.children) == 1:  # Only "Add Class" button remains
        class_edges_table.children = [create_class_row(1)] + list(class_edges_table.children)
        update_delete_buttons()

exposure_selector.observe(preview_impact_func, 'value')
approach_selector.observe(update_class_edges_table, 'value')

preview_chk = widgets.Checkbox(
    value=True,
    description='Preview Results',
    disabled=False,
    indent=False
)

# Button to run the script
run_button = widgets.Button(description="Run Analysis", layout=widgets.Layout(width='290px'), button_style='danger')
output = widgets.Output()

def validate_input():
    output.clear_output()
    
    # Check ISO_A3 code
    print("Checking country boundaries...")
    selected_country = country_selector.value
    iso_a3 = country_dict.get(selected_country)
    if not iso_a3:
        print(f"Error: {selected_country} is not a valid country selection.")
        return False

    url = "https://services.arcgis.com/iQ1dY19aHwbSDYIF/ArcGIS/rest/services/World_Bank_Global_Administrative_Divisions_VIEW/FeatureServer/5/query"
    params = {
        'where': f"ISO_A3 = '{iso_a3}'",
        'outFields': 'ISO_A3',
        'returnDistinctValues': 'true',
        'f': 'json'
    }
    response = requests.get(url, params=params)
    
    if response.status_code != 200:
        print("Error: Unable to validate country code. Please ensure you are connected to the internet.")
        return False
    data = response.json()
    if not data['features']:
        print(f"Error: {iso_a3} is not a valid ISO_A3 code.")
        return False
   
    # Check class thresholds
    if approach_selector.value == 'Classes':
        print("Checking class thresholds...")
        class_values = [child.children[1].value for child in class_edges_table.children[:-1]]  # Exclude "Add Class" button
        if len(class_values) > 1:
            
            is_seq = np.all(np.diff(class_values) > 0)
            if not is_seq:
                print("Error: Ensure that all class thresholds are ordered in ascending order.")
                return False

    print("User input accepted!")
    return True

def run_analysis_script(b):
    with output:
        if validate_input():
            # Gather input values
            selected_country = country_selector.value
            country = country_dict.get(selected_country)
            if not country:
                print("Error: Invalid country selection.")
                return
            
            # Rest of the function remains the same
            haz_cat = hazard_selector.value
            period = period_selector.value
            scenario = scenario_selector.value if period != '2020' else ''
            return_periods = list(return_periods_selector.value)
            exp_cat_list = list(exposure_selector.value)
            exp_year = '2020'  # This is fixed as per the original script
            adm_level = adm_level_selector.value
            analysis_type = approach_selector.value
            min_haz_slider = hazard_threshold_slider.value
            
            # Get class thresholds if applicable
            class_edges = [child.children[1].value for child in class_edges_table.children[:-1]] if analysis_type == 'Classes' else []  # Exclude "Add Class" button
            
            # Determine exp_nam_list
            if custom_exposure_radio.value == 'Custom exposure':
                exp_nam_list = [textbox.value for textbox in custom_exposure_container.children]
            else:
                exp_nam_list = [f"{country}_{exp}" for exp in exp_cat_list]

            # Additional parameters (you may want to add UI elements for these later)
            save_check_raster = False
            n_cores = None

            start_time = time.time()

            print(f"Starting analysis for {iso_to_country[country]} ({country})...")
            
            # Validate exp_nam_list
            if len(exp_nam_list) != len(exp_cat_list):
                print("ERROR: Parameter 'exp_nam_list' should have the same length as 'exp_cat_list'")
                return

            # Run analysis for each exposure category
            for i in range(len(exp_cat_list)):
                exp_cat = exp_cat_list[i]
                exp_nam = exp_nam_list[i]
                print(f"Running analysis for {exp_cat}...")
                result_df = run_analysis(country, haz_cat, period, scenario, return_periods, min_haz_slider,
                             exp_cat, exp_nam, exp_year, adm_level, analysis_type, class_edges, 
                             save_check_raster, n_cores)
                
            print(f"Analysis completed in {time.time() - start_time:.2f} seconds")
            # Plot results if preview is checked
            if preview_chk.value:
                print("Plotting results preview as map...")
                plot_results(result_df, country, adm_level, exp_cat, analysis_type)
  
run_button.on_click(run_analysis_script)

# Displaying the GUI
header_html = f"""
<div style='
    background: linear-gradient(to bottom, #003366, transparent);
    padding: 40px;
    border-radius: 10px 10px 0 0;
    display: flex;
    justify-content: space-between;
    align-items: flex-start;
    width: 100%;
'>
    <div align="center"><br>
        <h1 style='color: #FFFFFF; margin: 0;'>RISK DATA LIBRARY</h1>
        <h2 style='color: #ff5733; margin: 0;'><b>ANALYTICAL TOOL</b></h2><br>
        <h4 style='color: #118AB2; margin: 0;'><b>FLOOD HAZARD (FATHOM 3)</b></h2>
    </div>
    <img src="rdl_logo.png" style="width: 200px; height: auto;">
</div>
"""

header = widgets.HTML(value=header_html, layout=widgets.Layout(width='100%'))

# Layout the widgets in a form
form_items = [
    country_selector,
    adm_level_selector,
    hazard_selector,
    hazard_threshold_slider,
    period_selector,
    scenario_selector,
    return_periods_selector,
    exposure_selector,
    custom_exposure_radio,
    custom_exposure_container,
    approach_selector,
    approach_box,
]

form = widgets.VBox(form_items, layout=widgets.Layout(
    background='#003366', 
    padding='10px',
    border_radius='10px', 
    width='500px'
))

# Combine header and content in a vertical layout
right_box = widgets.VBox([info_box, preview_chk, run_button, output], layout=widgets.Layout(width='310px'))

# Create the layout with the info box on the right
content_layout = widgets.HBox([form, right_box], layout=widgets.Layout(width='100%', justify_content='space-between'))

# Combine header and content in a vertical layout
final_layout = widgets.VBox([header, content_layout], layout=widgets.Layout(width='800px'))

# JavaScript to handle hover events and update the info box
js_code = f"""
<script>
document.querySelector('.{country_selector_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Enter the country name.';
}};
document.querySelector('.{hazard_selector_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Select the type of flood hazard.\\nThe DEFENDED option accounts for protection standards set proportional to country GDP; it does not account for geolocated physical defence measures.';
}};
document.querySelector('.{hazard_threshold_slider_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Set the minimum hazard threshold, in the unit of the dataset.\\nValues below this threshold will be ignored.';
}};
document.querySelector('.{period_selector_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Select the reference time period for the hazard analysis:\\n\\n- Historical baseline: 2020\\n- Future periods: 2030, 2050, 2080';
}};
document.querySelector('.{scenario_selector_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Choose the climate scenario for future projections.';
}};
document.querySelector('.{return_periods_selector_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Choose the return periods to consider.\\nCtrl + click or drag mouse for multiple choices.\\n\\nReturn period defines the intensity of the hazard in relation to its occurrence probability.\\nWhen using the function approach, the whole range selection is preferred for best results.';
}};
document.querySelector('.{exposure_selector_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Select the exposure category on which the risk is calculated.\\nCtrl + click or drag mouse for multiple choices.';
}};
document.querySelector('.{custom_exposure_radio_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Choose between default exposure data or custom exposure data provided by the user.\\n\\nDefault data sources are:\\n- Population: WorldPop 2020 (100 m)\\n- Built-up: World Settlement Footprint 2019 (100 m)\\n- Agricolture: ESA World Cover 2019 - Crop area (1 km)';
}};
document.querySelector('.{custom_exposure_textbox_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Enter the filename (without extension) of your custom exposure data.\\n- Must be a valid .tif file using CRS:4326\\n- Must be located in the EXP folder. ';
}};
document.querySelector('.{approach_selector_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Select the approach for risk classification: Classes or Function.\\n\\n- Function approach is based on default impact functions for each exposure category.\\n- Classes approach measures the average annual exposure to user-defined hazard thresholds. Thresholds must be inserted in incremental order.';
}};
document.querySelector('.{adm_level_selector_id}').onmouseover = function() {{
    document.querySelector('.info-box textarea').value = 'Select the administrative level to aggregate and present the results of the analysis.';
}};
</script>
"""

# Display the layout and inject the JavaScript code
display(final_layout)
display(HTML(js_code))
# Initial update
update_custom_exposure_visibility()
update_class_edges_table()